In [25]:
# https://docs.opencv.org/3.4/d5/dc4/tutorial_adding_images.html


import numpy as np
import cv2 as cv2

In [27]:
# File
vid_file = "./video/cropped_trimmed.mp4"


In [28]:
# Output File
# I need to get the size of the input file to initialize the output file writer,
# to do this I have to capture one frame and analyze it.
cap_temp = cv2.VideoCapture(vid_file)
ret_temp, frame_temp = cap_temp.read()	
cap_temp.release()


# from: https://learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
# Define the fps to be equal to 10. Also frame size is passed.
 
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_temp.shape[1], frame_temp.shape[0]))



In [30]:
# Input Capture
cap = cv2.VideoCapture(vid_file)
# Initializing subtractor
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
#fgbg = cv2.createBackgroundSubtractorMOG2()
#fgbg = cv2.createBackgroundSubtractorKNN()
#fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()


while(1):
	key = cv2.waitKey(1) & 0xFF
	ret, frame = cap.read()	

	 # if frame is read correctly ret is True
	if not ret:
		print("Can't receive frame (stream end?). Exiting ...")
		break

	# applying on each frame
	mask = fgbg.apply(frame)
	#print(frame.shape, fgmask.shape)

	# apply mask
	masked = cv2.bitwise_and(frame, frame, mask=mask)
	
	# blend images
	alpha = 0.5
	beta = (1.0 - alpha)
	blended = cv2.addWeighted(frame, alpha, masked, beta, 0.0)

	# segment images
	alpha = 0
	beta = (1.0 - alpha)
	masked = cv2.addWeighted(frame, alpha, masked, beta, 0.0)


	# Display result
	cv2.imshow('blend', blended)
	cv2.imshow('frame', frame)
	cv2.imshow('mask', mask)
	cv2.imshow('masked', masked)

	# save output
	out.write(masked)

	# check for keyboard interruqqpt
	k = cv2.waitKey(1) & 0xff
	if k == 27:
		break
	elif key == ord("q"):
		break


cap.release()
out.release()
cv2.destroyAllWindows()


Can't receive frame (stream end?). Exiting ...


In [2]:
# Optical flow from https://docs.opencv.org/4.x/d4/dee/tutorial_optical_flow.html
import numpy as np
import cv2 as cv
#import argparse
#parser = argparse.ArgumentParser(description='This sample demonstrates Lucas-Kanade Optical Flow calculation. \
# The example file can be downloaded from: \
# https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4')
#parser.add_argument('image', type=str, help='path to image file')
#args = parser.parse_args()
cap = cv2.VideoCapture("./video/cropped_trimmed.mp4")
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
 qualityLevel = 0.3,
 minDistance = 7,
 blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize = (15, 15),
 maxLevel = 2,
 criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
 ret, frame = cap.read()
 if not ret:
    print('No frames grabbed!')
    break
 frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
 # calculate optical flow
 p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
 # Select good points
 if p1 is not None:
    good_new = p1[st==1]
    good_old = p0[st==1]
 # draw the tracks
 for i, (new, old) in enumerate(zip(good_new, good_old)):
    a, b = new.ravel()
    c, d = old.ravel()
    mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
    frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
 # Now update the previous frame and previous points
 old_gray = frame_gray.copy()
 p0 = good_new.reshape(-1, 1, 2)
cv.destroyAllWindows()

KeyboardInterrupt: 

In [3]:
# Dense Optical flow from https://docs.opencv.org/4.x/d4/dee/tutorial_optical_flow.html
import numpy as np
import cv2 as cv
cap = cv2.VideoCapture("./video/cropped_trimmed.mp4")
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
 ret, frame2 = cap.read()
 if not ret:
    print('No frames grabbed!')
    break
 next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
 flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
 mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
 hsv[..., 0] = ang*180/np.pi/2
 hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
 bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
 cv.imshow('frame2', bgr)
 k = cv.waitKey(30) & 0xff
 if k == 27:
    break
 elif k == ord('s'):
    cv.imwrite('opticalfb.png', frame2)
    cv.imwrite('opticalhsv.png', bgr)
 prvs = next
cv.destroyAllWindows()

KeyboardInterrupt: 